In [15]:
# Данные для демонстрации примеров отбора признаков
import pandas as pd
from sklearn.datasets import make_classification
X,y = make_classification(n_samples= 10_000, n_features=10, n_redundant=4, random_state=24675)
X = pd.DataFrame(columns=[f"x{i}" for i in range(X.shape[1])], data=X)

X.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,1.462224,-0.353898,0.045090,-0.790188,0.802854,0.450286,0.873850,0.872814,0.723634,0.888334
1,-0.690806,-0.677625,0.163398,0.249252,0.076161,0.777240,1.055399,0.583178,-0.136880,1.210272
2,-0.243481,1.995314,-0.749142,-0.459926,-0.472001,-1.993219,-0.959083,2.066811,0.045997,-1.857289
3,0.183242,-0.686938,0.278538,-0.243262,0.301167,0.663480,0.164824,-1.002783,-1.587861,0.508085
4,-0.107274,-1.228168,0.391958,0.576177,0.780719,1.303111,1.144787,-0.295717,1.020494,1.583554


# Отбор признаков (feature selection)

**Зачем?**
- Убирает проблему мультиколлинаераности (актуальна для некоторых моделей) 
- Упрощает модель, 
    - следовательно упрощает задачу обучения модели. Больше шансов построить более точную модель.
    - модель можно обучить быстрее
    - простые модели менее склонны к переобучению.

**проклятье размерности (curse of dimensionality)** - проблемы возникающие при большом количестве признаков:
- возможны случайные зависимости между признаками (в том числе между целевым и независимыми)
- повышенное потребление памяти 

**Ссылки**
1. https://scikit-learn.org/stable/modules/feature_selection.html


## Критерии отбора
#### 1. На основе изменчивости признаков
Удалять все константные признаки или признаки с маленьким стандартным отклонением.\
`from sklearn.feature_selection import VarianceThreshold`

#### 2. Фильтрационные методы - отбрасывать признаки по одному, независимо друг от друга
- По коэффициенту корреляции Пирсона (линейному к.к.).
    - Выбирать признаки, которые имеют сильную корреляцию с целевым признаком. Например $|r| > 0.2$
    - Отбросить те признаки, для которых есть парный признак с высокой корреляцией.\
    Это поможет избавиться от почти линейной зависимости между признаками, которая вредит некоторым моделям (линейная и логистическая регрессия).
- Другие показатели взаимосвязи признаков (например нелинейные К.К.). См. также пакет `phik`



#### 3. На основе показателей (или параметров) некоторых моделей (Model-based method)
- Коэффициенты перед признаками в линейной или логистической регрессии.\
    Чем больше коэффициент тем более значим признак. *Признаки должны быть приведены к одной и той же шкале*\
- Значимость признаков на основе случайного леса или градиентного бустинга.

Во время обучения дерева ([пример](https://colab.research.google.com/drive/1Bin_h7BPSfnxs4Pea7eibceNMkOEcKmg?usp=sharing)) нужно решать два вопроса: какой признак использовать для разделения узла на ветви, какую границу значения признака выбрать. 
Выбирается тот признак, который даёт больше прироста информации (позволяет создать более эффективное разделение). Для оценки этого показателя используются или коэффициент Джини или показатель информационной энтропии.
Чем чаще признак использовался для разделения, тем важнее признак. Обычно это количество делиться на общее количество деревьев (или узлов во всех деревьях)

В SKlearn есть специальный класс, который способен автоматически отбирать признаки, вне зависимости от того, какая модель предоставляет показатели важности признаков - `SelectFromModel`.


Такое измерение важности признаков, в отличии, например от К.К., позволяет оценить важность отдельного признака, но при этом все признаки учитываются в совокупности.

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

clf = RandomForestClassifier()
clf.fit(X, y)

print("feature_importances")
print( "  ".join( [f"x{i:<4}" for i in range(len(clf.feature_importances_))]) )
print( "  ".join( [f"{x:5.3f}" for x in clf.feature_importances_]) )

# отбирает все признаки, у которых вес (weight или показатель важности), полученный из модели МО, превышает установленное пороговое значение
feature_selector = SelectFromModel( clf, threshold="median", prefit=True)
# threshold = {median, mean, number} - порог отбора признаков
# max_features (default = None) - задаёт максимальное количество признаков, нужно задать threshold=-np.inf если использовать этот параметр
# prefit - используется уже обученная модель?
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html

# преобразуем данные
X_filtred = feature_selector.fit_transform(X, feature_names = X.columns)

print(f"\nРазмерность новых признаков: {X_filtred.shape[1]}")
feature_selector.get_feature_names_out()

feature_importances
x0     x1     x2     x3     x4     x5     x6     x7     x8     x9   
0.017  0.120  0.072  0.018  0.017  0.217  0.156  0.049  0.018  0.316

Размерность новых признаков: 5


array(['x1', 'x2', 'x5', 'x6', 'x9'], dtype=object)




#### 4. Встроенные методы (Embedded methods) - отбор признаков - часть процесса обучения модели
См. регуляризации (L1 или L2) в линейных моделях.

#### 5. (Wrapper methods - ручной или автоматизированный отбор признаков на основе оценки модели на тестовой выборке.

1. Обучить модель. Оценить на тестовой выборке. Удалить признак. Повторить обучение и оценку. Удалить признак окончательно если качество выросло. Повторить для всех признаков или комбинаций признаков.
2. Аналогичный алгоритм, но начинать с одного признака и добавлять признаки.\

```python
from sklearn.feature_selection import SequentialFeatureSelector
bfs = SequentialFeatureSelector(lr, k_features='best', forward = False, n_jobs=-1)
```
3. Исчерпывающий выбор признаков - полный перебор всех комбинаций признаков.
```python
from mlxtend.feature_selection import ExhaustiveFeatureSelector
```
- см. Recursive feature elimination


#### 5. Уменьшение размерности
PCA, UMAP и др. методы

In [1]:
from sklearn.decomposition import PCA
# создаем объект PCA
pca = PCA(n_components=2)
# обучаем модель и преобразуем данные
X_transformed = pca.fit_transform(X)


ModuleNotFoundError: No module named 'sklearn'

# Извлечение признаков (feature extraction)